In [ ]:
!pip install language-tool-python pandas openpyxl xlsxwriter
!apt-get -y update >/dev/null
!apt-get -y install openjdk-17-jre-headless >/dev/null

!update-alternative --install /usr/bin/java java /usr/lib/jvm/java-17-openjdk-amd-64/bin/java 1
!update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java
!java -version



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 5.6 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
/bin/bash: line 1: update-alternative: command not found
openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import language_tool_python as lt
from google.colab import files
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from datetime import datetime
import io
import os


In [ ]:
print("Suba el archivo .xlsx")
uploaded = files.upload()
assert len(uploaded) == 1 #Solo recibe un archivo
input_filename = list(uploaded.keys())[0]
assert input_filename.lower().endswith(".xlsx") #Obligatoriamente un .xlsx


Suba el archivo .xlsx


Saving indicadores_anexo_9.xlsx to indicadores_anexo_9.xlsx


In [ ]:
SHEET_NAME = "indicadores"
TARGET_COLUMNS = [
    "nombre_indicador",
    "descripcion",
    "formula",
    "numerador",
    "fuente_numerador",
    "denominador",
    "fuente_denominador",
    "observaciones",
    "metodologia",
    "exclusiones",
]

# Salidas
base, ext = os.path.splitext(input_filename)
output_filename = f"{base}_corregido.xlsx"
log_filename = f"{base}_log_correcciones.csv"

# ======= CARGUE DE DATOS (LIBRO Y HOJA) ==========


# Validacion para existencia del archivo y la hoja
try:
  wb = load_workbook(filename=input_filename)
except Exception as e:
  raise RuntimeError(f"No se pudo abrir el archivo. Verifique si es .xlsx. Detalle: {e}")

if SHEET_NAME not in wb.sheetnames:
  raise RuntimeError(f'La hoja "{SHEET_NAME}" no existe en el archivo. Hojas disponibles: {wb.sheetnames}')

ws = wb[SHEET_NAME]

# Detecteccion de encabezados\
headers = []
for cell in ws[1]:
  headers.append(str(cell.value).strip() if cell.value is not None else "")
print("columnas encontradas en la hoja")
for c in headers:
  print(" -", c)

# Mapear el nombre de la columna
header_to_colidx = {h: i+1 for i,h in enumerate(headers) if h}

#Validacion de las columnas objetivo (En este caso: {SHEET_COLUMNS})
present = [c for c in TARGET_COLUMNS if c in header_to_colidx]
missing = [c for c in TARGET_COLUMNS if c not in header_to_colidx]

print(f"\n Validacion de columnas objetivo: ")
print(" Presentes:", present if present else "Ninguna")
print(" Faltantes:", missing if missing else "Ninguna")

if not present:
  print("\n No se encontro ninguna de las columnas objetivo. No se realizan cambios")
  wb.save(output_filename)
  files.download(output_filename)
  raise SystemExit



columnas encontradas en la hoja
 - id_indicador
 - tipo_indicador
 - categoria
 - nombre_indicador
 - fecha_creacion_indicador
 - cod_res_256_de_2016
 - cod_propio
 - descripcion
 - formula
 - numerador
 - fuente_numerador
 - denominador
 - fuente_denominador
 - unidad_de_medida
 - meta
 - periodicidad
 - progresividad
 - observaciones
 - metodologia
 - exclusiones
 - responsable
 - cod_reps
 - grupo_indicador

 Validacion de columnas objetivo: 
 Presentes: ['nombre_indicador', 'descripcion', 'formula', 'numerador', 'fuente_numerador', 'denominador', 'fuente_denominador', 'observaciones', 'metodologia', 'exclusiones']
 Faltantes: Ninguna


In [ ]:
# Inicializacion del LanguageTool

print(" Iniciando LanguageTool para español")
tool = lt.LanguageTool('es')
print("LanguageTool Inicializado")



 Iniciando LanguageTool para español


INFO:language_tool_python.download_lt:Unzipping /tmp/tmpfbcusj2o.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://internal1.languagetool.org/snapshots/LanguageTool-latest-snapshot.zip to /root/.cache/language_tool_python.


LanguageTool Inicializado


In [ ]:
# Funcion de Correcion

def clean_text(x):
  """ Convierte a str los valores no nulos; ignora valores numericos o formulas. """
  if x is None:
    return None
  # Si es numero, fecha, bool, NO TOCAR
  if isinstance(x, (int, float, np.number)) or hasattr(x, "toordinal"):
    return x
  s = str(x)
  # Evita tocar las celdas que parecen formulas o empiezan por '='
  if s.startswith("="):
    return x
  # Evita string vacios o de solo espacios
  if s.strip() == "":
    return x
  return s

def apply_corrections(text):
  """ Aplica correcciones con LT """
  if not isinstance(text, str):
    return text, []
  matches = tool.check(text)
  corrected= lt.utils.correct(text,matches)
  return corrected, matches


In [ ]:
#LOGS DE CORRECCIONES

corrections_log  = [] # Detalles de las correciones
total_cells_visited = 0
total_cells_changed = 0

#Numero de filas usadas
max_row = ws.max_row

for col_name in present:
  col_idx = header_to_colidx[col_name]
  col_letter = get_column_letter(col_idx)

  for row_idx in range(2, max_row + 1):
    cell = ws.cell(row=row_idx, column=col_idx)
    original = cell.value
    processed = clean_text(original)
    total_cells_visited += 1

    # Saltar si no es texto o no hay nada que corregir
    if(not isinstance(processed, str)) or (processed.strip()==""):
      continue

    corrected, matches = apply_corrections(processed)

    if isinstance(corrected, str) and corrected != processed:
      #Registrar
      total_cells_changed += 1
      cell_address = f"{col_letter}{row_idx}"

      # Para los logs, se guardan las primeras 5 reglas aplicadas en la celda
      rule_details = []
      for m in matches[:5]:
        rule_details.append({
            "rule_id":m.ruleId,
            "message":m.message,
            "replacements":m.replacements[:3],
            "offset":m.offset,
            "length":m.errorLength,
        })

        corrections_log.append({
            "columna":col_name,
            "celda":cell_address,
            "original":processed,
            "corregido":corrected,
            "n_correcciones_detectadas": len(matches),
            "detalles_rules": rule_details,

        })

        cell.value = corrected




In [ ]:
#GUARDAR LOS RESULTADOS

wb.save(output_filename)

#Exportar los logs a csv
def flatten_rules(row):
  """Convierte la lista de reglas en texto legible para csv"""
  parts = []
  for i, r in enumerate(row.get("detalles_rules", []), start=1):
    rep = ", ".join(r.get("replacements",[])) if r.get("replacements") else ""
    parts.append(f"[{i}] {r.get('rule_id','')} | {r.get('message','')} | Sugerencias: {rep}")
  return " || ".join(parts)

if corrections_log:
  df_log = pd.DataFrame(corrections_log)
  df_log["resumen_reglas"] = df_log.apply(flatten_rules, axis=1)
  #Elegimos columnas clave para el CSV
  cols = ["columna", "celda", "n_correcciones_detectadas", "original", "corregido", "resumen_reglas"]
  df_log[cols].to_csv(log_filename, index=False, encoding="utf-8-sig")

print("Proceso Completado")
print(f" Celdas visitadas: {total_cells_visited}")
print(f" Celdas Modificadas: {total_cells_changed}")
print(f" Archivo Corregido: {output_filename}")

if corrections_log:
  print(f" Log de correcciones: {log_filename}")





Proceso Completado
 Celdas visitadas: 1280
 Celdas Modificadas: 383
 Archivo Corregido: indicadores_anexo_9_corregido.xlsx
 Log de correcciones: indicadores_anexo_9_log_correcciones.csv


In [ ]:
# Muestra en Pantalla

max_examples = 15
if corrections_log:
  print("\n Ejemplos de correcciones (max. 15):")
  for i, r in enumerate(corrections_log[:max_examples], start=1):
    print(f"\n[{i}] Columna: {r['columna']} | Celda: {r['celda']}")
    print(f"  -Antes:{r['original']}")
    print(f"  -Despues:{r['corregido']}")
    if r["detalles_rules"]:
      print("  -Reglas/sugerencias:")
      for rd in r["detalles_rules"]:
        sug = ", ".join(rd.get("replacements", [])) if rd.get("replacements") else "(sin sugerencias)"
        print(f"  {rd.get('rule_id','')} -> {rd.get('message','')} | Sugerencias: {sug}")



 Ejemplos de correcciones (max. 15):

[1] Columna: nombre_indicador | Celda: D10
  -Antes:Oportunidad En La Radicación De Facturas Por Proveedor  
  -Despues:Oportunidad En La Radicación De Facturas Por Proveedor 
  -Reglas/sugerencias:
  WHITESPACE_RULE -> Posible error tipográfico: múltiples espacios en blanco | Sugerencias:  

[2] Columna: nombre_indicador | Celda: D11
  -Antes:Oportunidad En La Gestion De Las Devoluciones
  -Despues:Oportunidad En La Gestión De Las Devoluciones
  -Reglas/sugerencias:
  MORFOLOGIK_RULE_ES -> Se ha encontrado un posible error ortográfico. | Sugerencias: Gestión, Gestiona, Gestionó

[3] Columna: nombre_indicador | Celda: D12
  -Antes:Oportunidad En La Respuesta De Glosas Por Proveedores  
  -Despues:Oportunidad En La Respuesta De Glosas Por Proveedores 
  -Reglas/sugerencias:
  WHITESPACE_RULE -> Posible error tipográfico: múltiples espacios en blanco | Sugerencias:  

[4] Columna: nombre_indicador | Celda: D15
  -Antes:Tiempo promedio de espera para

In [ ]:
# Descarga
print(" Descargando Archivos...")
files.download(output_filename)
if corrections_log:
  files.download(log_filename)

 Descargando Archivos...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>